In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import datetime
import matplotlib.pyplot as plt
#the points are taken from the Chapter 6 Delaunay Triangulation (Delaunay_Triangulations_ETH_Zuerich.pdf)
data_points =np.array([[1.5, 0.2], [4.4, 0.7], [3.1, 1],[0.2, 1.7], [2.2, 1.7], [3.8, 2], [1.4, 2.4], [4.3, 2.9], [2.5, 3.1], [1.2, 3.2]])



from scipy.spatial import Delaunay
import numpy as np


def alpha_shape(points, alpha, only_outer=True):
    """
    Compute the alpha shape (concave hull) of a set of points.
    :param points: np.array of shape (n,2) points.
    :param alpha: alpha value.
    :param only_outer: boolean value to specify if we keep only the outer border
    or also inner edges.
    :return: set of (i,j) pairs representing edges of the alpha-shape. (i,j) are
    the indices in the points array.
    """
    assert points.shape[0] > 3, "Need at least four points"

    def add_edge(edges, i, j):
        """
        Add a line between the i-th and j-th points,
        if not in the list already
        """
        if (i, j) in edges or (j, i) in edges:
            # already added
            assert (j, i) in edges, "Can't go twice over same directed edge right?"
            if only_outer:
                # if both neighboring triangles are in shape, it's not a boundary edge
                edges.remove((j, i))
            return
        edges.add((i, j))

    tri = Delaunay(points)
    edges = set()
    # Loop over triangles:
    # ia, ib, ic = indices of corner points of the triangle
    for ia, ib, ic in tri.vertices:
        pa = points[ia]
        pb = points[ib]
        pc = points[ic]
        # Computing radius of triangle circumcircle
        # www.mathalino.com/reviewer/derivation-of-formulas/derivation-of-formula-for-radius-of-circumcircle
        a = np.sqrt((pa[0] - pb[0]) ** 2 + (pa[1] - pb[1]) ** 2)
        b = np.sqrt((pb[0] - pc[0]) ** 2 + (pb[1] - pc[1]) ** 2)
        c = np.sqrt((pc[0] - pa[0]) ** 2 + (pc[1] - pa[1]) ** 2)
        s = (a + b + c) / 2.0
        area = np.sqrt(s * (s - a) * (s - b) * (s - c))
        circum_r = a * b * c / (4.0 * area)
        if circum_r < alpha:
            add_edge(edges, ia, ib)
            add_edge(edges, ib, ic)
            add_edge(edges, ic, ia)
    return edges



tri=Delaunay(data_points)
print(tri.vertices)
for a, b, c in tri.vertices:
    for i, j in [(a, b), (b, c), (c, a)]:
        plt.plot(data_points[[i, j], 0], data_points[[i, j], 1], color='k')
plt.savefig(f'./DelaunayTriangulation.png', transparent=True)        
plt.show()


# Computing the alpha shape
tri=Delaunay(data_points)
#print(tri.vertices)
#for a, b, c in tri.vertices:
 #   for i, j in [(a, b), (b, c), (c, a)]:
  #      plt.plot(data_points[[i, j], 0], data_points[[i, j], 1], color='gray')
#plt.show()
for alpha in [0.6, 0.7, 0.75,0.8,0.9,0.92,0.93,1,1.1,2,3]:
      #plot as a graph basis the DT in gray color
            
    for a, b, c in tri.vertices:
     for i, j in [(a, b), (b, c), (c, a)]:
         plt.plot(data_points[[i, j], 0], data_points[[i, j], 1], color='tab:gray')
    
    edges = alpha_shape(data_points, alpha=alpha, only_outer=True)
    #fig=plt.figure(figsize=(4,4))
    #ax=fig.add_subplot(2, 2, 2)
    #circle1 = plt.Circle((0, 0), 0.2, color='r')
    #ax.add_patch(circle1)
    for i, j in edges:
        plt.plot(data_points[[i, j], 0], data_points[[i, j], 1], color='r')#, label='alpha')
         # ax.legend(loc='best')
    plt.scatter(*zip(*data_points), color='r') #color=['b', 'g', 'r', 'c', 'm', 'y'])
    #ax.add_artist(circle1)
    plt.axis('off')
    plt.savefig(f'./a-shape_{alpha}.png', transparent=True)
    plt.show()
    